# Compression d'un texte
vincent.mazet@unistra.fr, 03/2022

In [1]:
import numpy as np
import komm

In [2]:
# Nombre de caractères
K = 16e3

# Taille de l'alphabet du canal (canal binaire)
N = 2

# Taille de l'alphabet de la source
M = 26 + 4
print(f"Taille de l'alphabet de la source   M = {M} symb")

# Taille de l'alphabet du support de stockage
print(f"Taille de l'alphabet du canal       N = {N:.0f} symb")

# Probabilité des caractères
probas = np.loadtxt("probas.csv", delimiter=",", skiprows=1, usecols=1)

# Entropie
H = komm.entropy(probas)
if H < np.log2(M):
    redondance = f"(< log2(M) = {np.log2(M):.2f} donc compression possible sans perte)"
else:
    redondance = f"(= log2(M) = {np.log2(M):.2f} donc compression impossible sans perte)"
print(f"Entropie de la source               H = {H:.2f} Sh/symb    {redondance}")

# Débit canal
Dc = 100e6
print(f"Débit canal                         Dc = {Dc/1e6:.2f} Mbit/s")

# Capacité canal
Cc = np.log2(N) * Dc
print(f"Capacité canal                      Cc = {Cc/1e6:.2f} Sh/µs")

# Taux de transmission maximal
Ts = Cc
print(f"Taux de transmission maximal        Ts = {Ts/1e6:.2f} Sh/µs")

# Débit de la source
Ds = Ts / H
print(f"Débit de la source                  Ds = {Ds/1e6:.2f} symb/µs\n")

# Temps minimal théorique nécessaire pour enregistrer le fichier sans erreur
tps = K / Ds
print(f"Temps minimal théorique nécessaire pour transmettre le fichier sans erreur : {tps*1e6:.2f} µs\n")

Taille de l'alphabet de la source   M = 30 symb
Taille de l'alphabet du canal       N = 2 symb
Entropie de la source               H = 4.09 Sh/symb    (< log2(M) = 4.91 donc compression possible sans perte)
Débit canal                         Dc = 100.00 Mbit/s
Capacité canal                      Cc = 100.00 Sh/µs
Taux de transmission maximal        Ts = 100.00 Sh/µs
Débit de la source                  Ds = 24.46 symb/µs

Temps minimal théorique nécessaire pour transmettre le fichier sans erreur : 654.22 µs



## Code ASCII

Le code ASCII est un code de longueur fixe, chaque symbole est codé sur 8 bits.
Donc la longueur moyenne est 8 bits/symb.

In [3]:
L = 8
T = K * L / Dc

print(f"Longueur moyenne : {L:.2f} bits/symb")
print(f"Durée de transmission : {T*1e6:.2f} µs")

Longueur moyenne : 8.00 bits/symb
Durée de transmission : 1280.00 µs


## Code de Huffman de 1<sup>re</sup> espèce

Le code peut être construit de la façon suivante :

In [4]:
code = komm.HuffmanCode(probas)
v = list(code.enc_mapping.values())

In [5]:
L = 0
for i in range(len(v)):
    L += code.pmf[i] * len(v[i])
T = K * L / Dc

print(f"Longueur moyenne : {L:.2f} bits/symb")
print(f"Durée de transmission : {T*1e6:.2f} µs")

Longueur moyenne : 4.12 bits/symb
Durée de transmission : 659.31 µs


## Code de Huffman de 2<sup>e</sup> espèce

Les probabilités des nouveaux symboles (combien y en a-t-il ?) peuvent être calculées ainsi :

In [6]:
probas2 = np.kron(probas,probas)

In [7]:
code = komm.HuffmanCode(probas2)
v = list(code.enc_mapping.values())
L = 0
for i in range(len(v)):
    L += code.pmf[i] * len(v[i])
L = L /2
T = K * L / Dc

print(f"Longueur moyenne : {L:.2f} bits/symb")
print(f"Durée de transmission : {T*1e6:.2f} µs")

Longueur moyenne : 4.10 bits/symb
Durée de transmission : 656.74 µs


Il ne faut pas oublier de diviser la longueur moyenne par 2 puisque la longueur moyenne calculée correspond à un alaphabet contenant des _couples_ de symboles. On observe ainsi que regrouper les symboles deux par deux pour constituer un nouvel alphabet permet de gagner un peu plus en temps de transmission.